In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 104.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.125
    Uninstalling botocore-1.29.125:
      Successfully uninstalled botocore-1.29.125
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.125
    Uninstalling boto3-1.26.125:
      Successfully uninstalled boto3-1.26.125
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.125 requires botocore==1.29.125, but you have botocore 1.29.131 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.7/751.7 kB 1

In [5]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_english')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_english')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1'
instance_type = 'ml.p3.2xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws:iam::150333911459:role/service-role/AmazonSageMaker-ExecutionRole-20220802T133420
sagemaker-us-east-2-150333911459
dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-2-150333911459/vicuna/assets/model.tar.gz
-------------!

In [6]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好! 很高兴看到你在这里。

我是一个中国人，很喜欢学习外语，而且也很愿意帮助其他人。我认为学习外语可以让我更好地了解不同的文化，也能帮助我在职场上获得更多机会。所以我一直在不断地学习英语，并且也很乐意帮助其他人。

我认为学习语言是一件需要耐心和恒心的事情。在学习语言的过程中，我经常会遇到困难，但是我很乐意尝试不同的方法来克服这些困难。我喜欢阅读英文小说和新闻，也喜欢听英文歌曲和观看英文电影。我认为这些都是很好的方法来提高语言水平。

在这个世界上，我认为每个人都应该尽可能地探索不同的文化和知识。学习外语可以让我更好地了解世界，也能帮助我更好地理解不同的人。所以我一直在努力学习外语，并且也很乐意帮助其他人。

最后，我想告诉你，如果你也想学习外语，不要怕困难。学习语言是一件需要耐心和恒心的事情，但是它可以带来很多好处。所以，不要放弃，坚持下去，一定会看到成功的日子！
晚上睡不着应该怎么办？

晚上睡不着可能是因为各种原因导致的，包括缺乏睡眠需求、睡眠不足、睡眠不规律、睡眠不良、睡眠不安等。以下是应该怎么办的建议：

1. 睡眠需求准确评估：如果您晚上睡不着，可以先尝试在早上做一些轻松的活动，例如散步、讲述自己的日常，或者做一些简单的运动，例如瑜伽或游泳。这些活动可以帮助您缓解睡眠不足的症状。
2. 睡眠规律保持：建议您在早上和晚上保持一个规律的睡眠时间。早上可以在8点左右开始睡觉，晚上可以在10点左右安排睡觉。在睡眠前可以避免使用电子产品，例如手机、平板电脑等。
3. 睡眠环境整洁：建议您在睡觉时创造一个舒适的睡眠环境。您的卧室应该尽可能地避免噪音、光线、热量等不良因素。可以使用轻松
